# Model Architecture

In [10]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from transformers.modeling_outputs import SequenceClassifierOutput


class BertConcatClassifier(nn.Module):
    def __init__(self, bert_model_name, num_labels):
        super(BertConcatClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(bert_model_name, output_hidden_states=True)
        self.num_labels = num_labels
        #classification part
        self.conv = nn.Conv1d(in_channels=3, out_channels=1, kernel_size=1)  # 3x768 -> 1x768
        self.relu = nn.ReLU()
        self.linear = nn.Linear(768, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        hidden_states = outputs.hidden_states

        last_cls_vector = hidden_states[-1][:, 0, :]
        fourth_last_cls_vector = hidden_states[-4][:, 0, :]

        attention_mask_expanded = attention_mask.unsqueeze(-1).expand_as(hidden_states[-1])
        masked_last_layer = hidden_states[-1][:, 1:, :] * attention_mask_expanded[:, 1:, :]
        mean_pooled_vector = masked_last_layer.mean(dim=1)

        concatenated_vector = torch.cat((last_cls_vector.unsqueeze(1), fourth_last_cls_vector.unsqueeze(1), mean_pooled_vector.unsqueeze(1)), dim=1)
        # print("concatenated_vector shape: ", concatenated_vector.shape)


        conv_output = self.conv(concatenated_vector).squeeze(2)
        conv_output = self.relu(conv_output)
        # print("conv_output shape: ", conv_output.shape)

        logits = self.linear(conv_output)
        logits = logits.squeeze(1)
        # print("logits shape: ", logits.shape)

        # 손실 계산
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


# Model에 config 특성 추가

In [11]:
import torch
import torch.nn as nn
from transformers import AutoModel, PreTrainedModel, BertConfig, BertModel
from transformers.modeling_outputs import SequenceClassifierOutput
import json

# config_bert = BertConfig(
#     vocab_size=32000,  # 예시 vocab 크기
#     hidden_size=768,
#     num_hidden_layers=12,
#     num_attention_heads=12,
#     intermediate_size=3072,
#     output_hidden_states=True
# )

class BertConcatClassifier(PreTrainedModel):
    def __init__(self, config):
        super(BertConcatClassifier, self).__init__(config)
        self.config = config
        self.bert = AutoModel.from_pretrained("klue/bert-base", output_hidden_states=True)
        self.num_labels = config.num_labels
        
        # classification part
        self.conv = nn.Conv1d(in_channels=config.conv_in_channels, out_channels=config.conv_out_channels, kernel_size=config.conv_kernel_size)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(config.linear_in_features, self.num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask,return_dict=True)
        hidden_states = outputs.hidden_states

        last_cls_vector = hidden_states[-1][:, 0, :]
        fourth_last_cls_vector = hidden_states[-4][:, 0, :]
        
        attention_mask_expanded = attention_mask.unsqueeze(-1).expand_as(hidden_states[-1])
        masked_last_layer = hidden_states[-1][:, 1:, :] * attention_mask_expanded[:, 1:, :]
        mean_pooled_vector = masked_last_layer.mean(dim=1)

        concatenated_vector = torch.cat((last_cls_vector.unsqueeze(1), fourth_last_cls_vector.unsqueeze(1), mean_pooled_vector.unsqueeze(1)), dim=1)

        conv_output = self.conv(concatenated_vector).squeeze(2)
        conv_output = self.relu(conv_output)

        logits = self.linear(conv_output)
        logits = logits.squeeze(1)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        # return logits
        
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions
        )
        
    def save_config(self, save_path):
        with open(save_path, 'w') as f:
            json.dump(self.config.to_dict(), f, indent=2)
            
    @classmethod
    def from_pretrained(cls, config_path, **kwargs):
        """저장된 설정 파일로부터 모델을 로드합니다."""
        with open(config_path, 'r') as f:
            config_dict = json.load(f)
        config = BertConcatClassifier(**config_dict)
        return cls(config, **kwargs)


# Config file 설정

In [12]:
from transformers import BertConfig
import json

config = BertConfig(
    num_labels=60,                        # 예시 라벨 수
    conv_in_channels=3,                # BERT 마지막 히든 레이어 차원
    conv_out_channels=1,               # 출력 채널 수
    conv_kernel_size=1,                  # 커널 크기
    linear_in_features=768,               # 선형 계층 입력 피처 수
    vocab_size = 32000,
    
)
# with open('struc3_config.json', 'w') as json_file:
#     json.dump(config, json_file, indent=4)

# config_path = 'struc3_config.json'

# config = BertConfig.from_json_file(config_path)


# Model load with config

In [13]:
import torch.nn.functional as F
from safetensors.torch import load_file as safe_load_file
import torch
from transformers import AutoModel, AutoTokenizer

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

model = BertConcatClassifier(config)
model.to(device)

# checkpoint-18800에서 모델 가중치 로드 (safetensors 사용)
checkpoint_path = "/Users/kimduhyeon/Desktop/Huggingface/klue-bert-base-senti/checkpoint-8000/model.safetensors"
state_dict = safe_load_file(checkpoint_path)
model.load_state_dict(state_dict)

# 모델을 평가 모드로 설정
model.eval()


BertConcatClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [14]:
id2label = {
    0: "분노",
    1: "툴툴대는",
    2: "좌절한",
    3: "짜증내는",
    4: "방어적인",
    5: "악의적인",
    6: "안달하는",
    7: "구역질 나는",
    8: "노여워하는",
    9: "성가신",
    10: "슬픔",
    11: "실망한",
    12: "비통한",
    13: "후회되는",
    14: "우울한",
    15: "마비된",
    16: "염세적인",
    17: "눈물이 나는",
    18: "낙담한",
    19: "환멸을 느끼는",
    20: "불안",
    21: "두려운",
    22: "스트레스 받는",
    23: "취약한",
    24: "혼란스러운",
    25: "당혹스러운",
    26: "회의적인",
    27: "걱정스러운",
    28: "조심스러운",
    29: "초조한",
    30: "상처",
    31: "질투하는",
    32: "배신당한",
    33: "고립된",
    34: "충격 받은",
    35: "가난한, 불우한",
    36: "희생된",
    37: "억울한",
    38: "괴로워하는",
    39: "버려진",
    40: "당황",
    41: "고립된(당황한)",
    42: "남의 시선을 의식하는",
    43: "외로운",
    44: "열등감",
    45: "죄책감",
    46: "부끄러운",
    47: "혐오스러운",
    48: "한심한",
    49: "혼란스러운(당황한)",
    50: "기쁨",
    51: "감사하는",
    52: "신뢰하는",
    53: "편안한",
    54: "만족스러운",
    55: "흥분",
    56: "느긋",
    57: "안도",
    58: "신이 난",
    59: "자신하는"
}

In [15]:
id2label_text = {
    "0": "분노",
"1": "툴툴대는",
"2": "좌절한",
"3": "짜증내는",
"4": "방어적인",
"5": "악의적인",
"6": "안달하는",
"7": "구역질 나는",
"8": "노여워하는",
"9": "성가신",
"10": "슬픔",
"11": "실망한",
"12": "비통한",
"13": "후회되는",
"14": "우울한",
"15": "마비된",
"16": "염세적인",
"17": "눈물이 나는",
"18": "낙담한",
"19": "환멸을 느끼는",
"20": "불안",
"21": "두려운",
"22": "스트레스 받는",
"23": "취약한",
"24": "혼란스러운",
"25": "당혹스러운",
"26": "회의적인",
"27": "걱정스러운",
"28": "조심스러운",
"29": "초조한",
"30": "상처",
"31": "질투하는",
"32": "배신당한",
"33": "고립된",
"34": "충격 받은",
"35": "가난한, 불우한",
"36": "희생된",
"37": "억울한",
"38": "괴로워하는",
"39": "버려진",
"40": "당황",
"41": "고립된(당황한)",
"42": "남의 시선을 의식하는",
"43": "외로운",
"44": "열등감",
"45": "죄책감",
"46": "부끄러운",
"47": "혐오스러운",
"48": "한심한",
"49": "혼란스러운(당황한)",
"50": "기쁨",
"51": "감사하는",
"52": "신뢰하는",
"53": "편안한",
"54": "만족스러운",
"55": "흥분",
"56": "느긋",
"57": "안도",
"58": "신이 난",
"59": "자신하는"
}

# Inference

### id2label

## Inference

In [16]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

# 입력 텍스트 정의
input_texts = ["""
너 없는 지금도 눈부신 하늘과
눈부시게 웃는 사람들
나의 헤어짐을 모르는 세상은
슬프도록 그대로인데
시간마저 데려가지 못하게
나만은 널 보내지 못했나봐
가시처럼 깊게 박힌 기억은
아파도 아픈줄 모르고
그대 기억이 지난 사랑이
내 안을 파고 드는 가시가 되어
제발 가라고 아주 가라고
애써도 나를 괴롭히는데
아픈만큼 너를 잊게 된다면
차라리 앓고 나면 그만인데
가시처럼 깊게 박힌 기억은
아파도 아픈 줄 모르고
제발 가라고 아주 가라고
애써도 나를 괴롭히는데
너무 사랑했던 나를
크게 두려웠던 나를
미치도록 너를 그리워했던
날 이제는 놓아줘
보이지 않아 내 안에 숨어
잊으려 하면 할수록 더 아파와
제발 가라고 아주 가라고
애써도 나를 괴롭히는데
""", """옛날 옛날에 남양주시 와부읍
덕소리에서 자란 한 소년이 있었어요
아이는 주머니에 아무것도 없었지만
어머니의 사랑은 가득해
항상 웃음 짓던 소년이였죠
어느 날 그 소년 앞에
금을 두른 부자가 나타났답니다
부자는 그에게 물었어요
너에게 미래를 줄 테니
지금 이 순간을 나와 바꾸지 않겠니?
나 평생 꿈만을 꿨죠
알잖아요 꿈은 안 들잖아 돈
나 이 순간을 나 평생 떠올렸죠
친구들이 대학을 갈 때
난 한강에 가서 술을 마셨네
되뇌이면서 '세상은 날 싫어해'
그렇지 그렇지 그럴만했어
그때는 몰라 그리고 애써
알려고 하지도 않잖아
눈물을 흘렸지 내 방 속에서
눈물 흘려 여의도에서
두 눈물 맛이 달라 아
빌었어
빌었어 밤마다
이럴 땐 술김에 오그라들게
두 손을 모으고 말야
Oh oh 빌었어
빌었어 밤마다
나 무교잖아
근데 하늘에다가
비는 걸 보면 있나 봐
내 소원을 들어줄 어떤 이 (hey)""","""
니가 없는 거리에는 내가 할 일이 없어서
마냥 걷다 걷다 보면 추억을 가끔 마주치지
떠오르는 너의 모습 내 살아나는 그리움 한 번에
참 잊기 힘든 사람이란 걸 또 한 번 느껴지는 하루
어디쯤에 머무는지 또 어떻게 살아가는지
걷다 보면 누가 말해줄 것 같아
이 거리가 익숙했던 우리 발걸음이 나란했던
그리운 날들 오늘 밤 나를 찾아온다
널 그리는 널 부르는 내 하루는
애태워도 마주친 추억이 반가워
날 부르는 목소리에 돌아보면
텅 빈 거리 어느새 수많은 니 모습만 가득해
막다른 길 다다라서 낯익은 벽 기대 보면
가로등 속 환히 비춰지는 고백하는 니가 보여
떠오르는 그때 모습 내 살아나는 설레임 한 번에
참 잊기 힘든 순간이란 걸 또 한 번 느껴지는 하루
아직 나를 생각할지 또 그녀도 나를 찾을지
걷다 보면 누가 말해줄 것 같아
이 거리가 익숙했던 우리 발걸음이 나란했던
그리운 날들 오늘 밤 나를 찾아온다
널 그리는 널 부르는 내 하루는
애태워도 마주친 추억이 반가워
날 부르는 목소리에 돌아보면
텅 빈 거리 어느새 수많은 니 모습만 가득해
부풀은 내 가슴이 밤하늘에 외쳐본다
이 거리는 널 기다린다고
널 그리는 널 부르는 내 하루는
애태워도 마주친 추억이 반가워
날 부르는 목소리에 돌아보면
텅 빈 거리 어느새 수많은 니 모습만
가득해
""", """
아침에 눈을 떴을 때 너를
길을 걷다 멍하니 너를
지금은 내 곁에 없는 너를
그리워하네 바보처럼
나보다 행복하기를 바래
내 생각하지 않기를 바래
더 좋은 사람 만나길 바래
다시는 내게 올 수 없게
안개처럼 사라져 간 다시 못 올 그 지난날
함께한 추억 모두 흘려보낼게
널 잊어야 해 힘들어도
널 지워야 해 기억 속에서
네가 떠난 후에 난 죽을 것같이 아파도
두 번 다시 울지 않을게
잊을게 잊을게
아직도 휴대폰에 네 이름
지우지도 못하고 있어
전화기 들고 한참을 서서
널 생각하네 바보처럼
안개처럼 사라져 간 다시 못 올 그 지난날
함께한 추억 모두 흘려보낼게
널 잊어야 해 힘들어도
널 지워야 해 기억 속에서
네가 떠난 후에 난 죽을 것같이 아파도
다시는 너를 찾지 않아
아침에 눈을 떴을 때 너를 아직도
길을 걷다 멍하니 너를
지금은 내 곁에 없는 너를
그리워하네 바보처럼

""", """
너와 나 둘이 정신없이 가는 곳
정처 없이 가는 곳, 정해지지 않은 곳
거기서 우리 (whoa-whoa) 서로를 재워주고 (whoa-whoa)
서로를 깨워주고 (whoa-whoa) 서로를 채워주고 (whoa-whoa)
Excuse me 잠시만 아직까진 우린 남
하지만 조만간 중독성을 자랑하는 장난감
지금 이 느낌적인 느낌이 통하는 느낌
녹아버릴 아이스크림
지금이 우리에게는 꿈이야
너와 나 둘이서 추는 춤이야
기분은 미친 듯이 예술이야
Ooh-eh-oh, ooh-eh-oh, ooh-eh-oh
하늘을 날아가는 기분이야
죽어도 상관없는 지금이야
심장은 터질 듯이 예술이야
Ooh-eh-oh, ooh-eh-oh, ooh-eh-oh
예술이야 (예술이야), 예술이야 (예술이야)
예술이야 (예술이야), 이런 날이 올 줄이야
예술이야 (예술이야), 예술이야 (예술이야)
Ooh-eh-oh, ooh-eh-oh, ooh-eh-oh
너와 나 둘이 (whoa-whoa) 밤새 잔을 부딪혀 (whoa-whoa)
밤새 뺨을 부비며 (whoa-whoa) 밤새도록 둘이서 (whoa-whoa)
눈이 점점 풀린다
니도 따라 풀린다
끌린다, 너에 대한 수수께끼가 풀린다
지금 이 춤에 너의 가빠진 숨에
수줍음에 you know what I mean
지금이 우리에게는 꿈이야
너와 나 둘이서 추는 춤이야
기분은 미친 듯이 예술이야
Ooh-eh-oh, ooh-eh-oh, ooh-eh-oh
하늘을 날아가는 기분이야
죽어도 상관없는 지금이야
""","""
널 이제 놓아줘야 될것 같애
의미없는 날짜를 셀것 같애
내 손이 아닌 꽃이 무대인 나비
다시 날갯짓 하길
이제는 bye, bae
나에게는 크리스탈 혹은 별과 비슷한
빛을가진 diamond luxury, thang
널 봐 난
널 위한 건까? 날 위한 건지?
허나 넌 더욱 어울려 누군가의
더큰 어깨 제발 글로 갈래?
내가 나쁠래 모자 쓰고 말해
찡그리며 '오지 마'
너와 함께인 순간은 불안의
연속에 연속 내가 부족하니
미안해, 나의 자존심때문에
우아한 프라다
우아한 샤넬
깔끔한 마르지엘라
같은 너에 반해
잘하려 했지
허나 자격지심 때문에
너를 못가져 너를 못 안어
그댄 변함없이 아주 너무나도 아름다워
Run, run
Run, run
누가 먹였니? 너의 높은 값어칠
언제부터 너는 그리 아름다웠니
누가 넌 과분하게 나는 말없이, 그래
허나 너에겐 가면쓰고 감췄지
남겨 미안해 키스 같은 상처
그렇지않음 내가
별거 없는 애처럼
느껴져 몰아세운거야 미워해줘 날
모두 사랑해준 너
이제는 알겠어 내가 널 가진 것이
아니고 너가 와준것이라는 것을
너 없는 이 도시 마치 압구정의 club
이 속에 나는 먼지 나는 특별하지 않네
너를 위안 삼았나봐 너가 사랑할때
우리 걷던 거리 지나갈때
초라한 내게 그댄 빛을 발해
우아한 프라다
우아한 샤넬
깔끔한 마르지엘라
같은 너에 반해
잘하려 했지
허나 자격지심 때문에
너를 못가져 너를 못 안어
그댄 변함없이 아주 너무나도 아름다워
""","""
어둠만이 나의 전부였던 동안
숨이 벅차도록 달려왔잖아
Never say "time's up"
경계의 끝자락
내 끝은 아니니까
울타리 밖에 일렁이는 무언가
그 아무도 모르는 별 일지 몰라
I wanna wanna be there
I'm gonna gonna be there
벅찬 맘으로 이 궤도를 벗어나
Let's go!
새로운 길의 탐험가
Beyond the road
껍질을 깨뜨려버리자
두려움은 이제 거둬
오로지 나를 믿어
지금이 바로 time to fly
두 눈앞의 끝 사뿐 넘어가
한계 밖의 trip, 짜릿하잖아
녹이 슨 심장에 쉼 없이 피는 꿈
무모 하대도 믿어 난
누구도 본 적 없는 낯선 우주 속에
겁 없이 뛰어들어 fall (fall) fall (fall)
답답한 가슴 안에 불꽃을 피워낼래
Shine and bright
곧 잡힐 듯이 반짝이던 무언가
꼭 달릴수록 멀어져도 난 좋아
I never never give up
I'm getting getting better
여정은 이미 시작된 지 오래야
Let's go!
끝이라 생각한 순간
Beyond the road
넓은 세상이 날 감싸 안아
때로는 느릿해도 가끔은 지친대도
멈추지 않고 let me fly
두 눈앞의 끝 사뿐 넘어가
한계 밖의 trip, 짜릿하잖아
녹이 슨 심장에 쉼 없이 피는 꿈
무모 하대도 믿어 난
Woah woah woah
누가 뭐래도 믿어 난
Go, 다치고 망가져 버거워진 항해
Go, 숨 한 번 고르고 이어가면 okey
구름 너머 세상을 내 품에 안을래
두 눈앞의 끝 사뿐 넘어가
한계 밖의 trip, 짜릿하잖아
녹이 슨 심장에 쉼 없이 피는 꿈
무모 하대도 믿어 난
Woah woah woah
나의 여정을 믿어 난
""","""
바람 불어와 내 맘 흔들면
지나간 세월에
두 눈을 감아본다
나를 스치는 고요한 떨림
그 작은 소리에
난 귀를 기울여 본다
내 안에 숨쉬는
커버린 삶의 조각들이
날 부딪혀 지날 때
그 곳을 바라보리라
우리의 믿음 우리의 사랑
그 영원한 약속들을
나 추억한다면 힘차게 걸으리라
우리의 만남 우리의 이별
그 바래진 기억에
나 사랑했다면 미소를 띄우리라
내 안에 있는
모자란 삶의 기억들이
날 부딪혀 지날 때
그 곳을 바라보리라
우리의 믿음 우리의 사랑
그 영원한 약속들을
나 추억한다면 힘차게 걸으리라
우리의 만남 우리의 이별
그 바래진 기억에
나 사랑했다면 미소를 띄우리라
""","""
작은 연못에서 시작된 길
바다로 바다로 갈 수 있음 좋겠네
어쩌면 그 험한 길에 지칠지 몰라
걸어도 걸어도 더딘 발걸음에
너 가는 길이 너무 지치고 힘들 때
말을 해줘 숨기지마 넌 혼자가 아니야
우리도 언젠가 흰수염고래처럼 헤엄쳐
두려움 없이 이 넓은 세상 살아 갈 수 있길
그런 사람이길
더 상처 받지마 이젠 울지마 웃어봐
너 가는 길이 너무 지치고 힘들 때
말을 해줘 숨기지마 넌 혼자가 아니야
우리도 언젠가 흰수염고래처럼 헤엄쳐
두려움 없이 이 넓은 세상 살아 갈 수 있길
그런 사람이길
너 가는 길이 너무 지치고 힘들 때
말을 해줘 숨기지마 넌 혼자가 아니야
우리도 언젠가 흰수염고래처럼 헤엄쳐
두려움 없이 이 넓은 세상 살아 갈 수 있길
그런 사람이길
그런 사람이길
""","""
울려댔어 사이렌
텅 빈 길거린엔
도망치다 흘린 칼자루와 피가 흥건해
우리 그때 어릴땐
뭘 몰랐었지 Man
그냥 힘쎈 형이 제일로 멋졌었지 그땐
그래 우린 살아나왔어 지옥
이제 어딜가든 다 비옥
수도 없이 맛본 치욕
어릴때부터 입에 붙은 쌍욕
절대 할 수 없었지 신고
할 수 있는게 오직 기도
어떻게 느끼겠어 피곤
붉게 물들지 않을려 내 흰옷
아무 방법이 없어 No way
돈만 준다면 해 노예
내 믿음이 바뀌기 전에
주변이 다 바뀌길 원해
아직 죽지마 Ma bro
달려왔어 짓밟고
상처투성이된 몸
씻기엔 피가 물든 손
빈속에 피워대기엔 뻑뻑해
눈뜨고 나니 전봇대
일을 해도 Don't payback
가난한게 Make me 죄인
닥쳐 You know ma pain
편히자 Now we safe
벌어들여 다 Stack my chips
남들앞에 마침내
울려댔어 사이렌
텅 빈 길거린엔
도망치다 흘린 칼자루와 피가 흥건해
우리 그때 어릴땐
뭘 몰랐었지 Man
그냥 힘쎈 형이 제일로 멋졌었지 그땐
그래 우린 살아나왔어 지옥
이제 어딜가든 다 비옥
수도 없이 맛본 치욕
어릴때부터 입에 붙은 쌍욕
절대 할 수 없었지 신고
할 수 있는게 오직 기도
어떻게 느끼겠어 피곤
붉게 물들지 않을려 내 흰옷
진흙탕에서 STARTED
바닥에서부터 클라이밍
닥치는대로 다 TRY SHIT
돈을 원했어 단지
성공을 할수 있다면 고민 없이 걸어 내 모가지
이젠 눈물없이 보지 못하겠어 내 어릴적 사진
엄마 내가 돈 못벌면 파 호적
차라리 죽는게 나으니
얼른 가난을 졸업하고말꺼야
난 반드시 변하지않아 나의 목적
평생 한길만 팠으니
멈추지않아 나의 도전
난 잃을게 없는 하층민
나의 친구들의 눈엔
아직 슬픔이 담긴 흰자
내 친구들을 꼰지를
빠엔 앉아 전기의자
어릴적에 생긴 상처들을 다
명품옷으로 위장
내가 죽는 한이 있더라도
절대 잃지않아 희망
울려댔어 사이렌
텅 빈 길거린엔
도망치다 흘린 칼자루와 피가 흥건해
우리 그때 어릴땐
뭘 몰랐었지 Man
그냥 힘쎈 형이 제일로 멋졌었지 그땐
그래 우린 살아나왔어 지옥
이제 어딜가든 다 비옥
""","""
떨리는 너의 입술을 난 난 처음 보았지
무슨 말 하려고 말 하려고 오 뜸만 들이는지
슬픈 예감은 다 맞는단 노래 가사처럼
설마 아니겠지 아닐꺼야 오 아니어야만 돼
벌써 넌 나를 떠나 니 마음마저 떠나 또 몸마저도 떠나는데
난 몰라 널 잡을 방법을 좀 누가 내게 말해줘요
오늘밤 그 말만은 말아요 왜 날 버리고 갔나요
나 마음이 아파 가슴이 아파 눈물 차올라요
아직은 안녕 우린 안돼요
넌 그 입을 더 열지마
안녕이라고 내게 말하지마
차가운 너의 한마디가 날 주저앉혔지
세상 무너질 듯 무너진 듯 오 눈물만 눈물만
지금 순간이 가면 이 순간이 지나면 영영 우린 이별인데
사랑해 죽도록 사랑한 날 날 버리지 말아줘요
오늘밤 그 말만은 말아요 왜 날 버리고 갔나요
나 마음이 아파 가슴이 아파 눈물 차올라요
아직은 안녕 우린 안돼요 넌 그 입을 더 열지마
안녕이라고 내게 말하지마
이별이 뭔지 나는 몰라요 그냥 서럽고 서러워
나 사연이 많아 추억이 많아 가슴 찢어져요
아직은 안녕 우린 안돼요 넌 그 입을 더 열지마
안녕이라고 내게 말하지마 안녕이라고 내게 말하지마
""", """
참 잘하는 짓이다
""","""
파란 하늘 위로 훨훨 날아가겠죠
어려서 꿈꾸었던 비행기 타고
기다리는 동안 아무 말도 못 해요
내 생각 말할 순 없어요
모든 준비 다 끝났어 곱게 차려 입고 나선
바깥 풍경마저 들뜬 기분 때가 왔어
하늘 위로 날으는 순간이야 조금은 두려워도
애써 내색할 순 없어 이번이 처음이지만
전에 자주 비행했었잖아 친구들과 말썽장이
거북이 비행기로 올라탈 준비됐나
수많은 사람들 속을 지나쳐 마지막 게이트야
나도 모르게 안절부절 하고 있어
이럴 땐 침착해 좀 자연스럽게
파란 하늘 위로 훨훨 날아가겠죠
어려서 꿈꾸었던 비행기 타고
기다리는 동안 아무 말도 못 해요
내 생각 말할 순 없어요 yes remember
Yes fly 다들 아무 일도 없는 듯
하늘을 나르는데 아무 걱정 없는 듯
왠지 철딱서니 없었나 문득
이런 내 모습 촌스러워 입 다문 듯
쳐다보지 말아요
다들 처음 탈 때 이러지 않았나요
딴 데 봐요 신경 쓰지 마요
나 혼자 이런 게 나 좋아요
어떤 느낌일까 정말 새들처럼 나는 기분
세상 모든 것이 점처럼 보여지겠지 개구쟁이
거북이 비행기로 드디어 출발한다
수많은 사람들 속을 지나쳐 마지막 게이트야
나도 모르게 안절부절하고 있어
이럴 땐 침착해 좀 자연스럽게
파란 하늘 위로 훨훨 날아가겠죠
어려서 꿈꾸었던 비행기 타고
기다리는 동안 아무 말도 못 해요
내 생각 말할 순 없어요
기다리는 동안 아무 말도 못 해요
내 생각 말할 순 없어요 yes remember
""","""
마음 울적한 날엔 거리를 걸어보고
향기로운 칵테일에 취해도보고
한편의 시가 있는 전시회장도 가고
밤새도록 그리움에 편질 쓰고파
모차르트 피아노 협주곡 이십일번
그 음악을 내 귓가에 속삭여주며
아침 햇살 눈부심에 나를 깨워줄
그런 연인이 내게 있으면
나는 아직 순수함을 느끼고 싶어
어느 작은 우체국 앞 계단에 앉아
프리지아 꽃 향기를 내게 안겨줄
그런 연인을 만나 봤으면
마음 울적한 날엔 거리를 걸어보고
향기로운 칵테일에 취해도보고
한편의 시가 있는 전시회장도 가고
밤새도록 그리움에 편질 쓰고파
모차르트 피아노 협주곡 이십일번
그 음악을 내 귓가에 속삭여 주며
아침 햇살 눈부심에 나를 깨워줄
그런 연인이 내게 있으면
나는 아직 순수함을 느끼고 싶어
어느 작은 우체국 앞 계단에 앉아
프리지아 꽃향기를 내게 안겨줄
그런 연인을 만나 봤으면
마음 울적한 날엔 거리를 걸어보고
향기로운 칵테일에 취해도보고
한편의 시가 있는 전시회장도 가고
밤새도록 그리움에 편질 쓰고파
창밖에는 우울한 비가 내리고 있어
내 마음도 그 비따라 우울해지네
누가 내게 눈부신 사랑을 가져 줄까
이 세상은 나로 인해 아름다운데
마음 울적한 날엔 거리를 걸어보고
향기로운 칵테일에 취해도보고
한편의 시가 있는 전시회장도 가고
밤새도록 그리움에 편질 쓰고파
""","""
알잖아 너를 이토록
사랑하며 기다린 나를
뭐가 그리 바쁜지
너무 보기 힘들어
너 도대체 뭐하고 다니니
그게 아냐 이유는 묻지마
그냥 믿고 기다려 주겠니
내게도 사랑을 위한
시간이 필요해
널 받아들일 수 있게
일부로 피하는 거니 노
삐삐쳐도 아무 소식 없는 너
싫으면 그냥 싫다고
솔직하게 말해봐
말리지마 내 이런 사랑을
너만 보면 미칠 것 같은 이맘을
누가 알겠어 웨딩드레스 입은
니 곁에 다른 사람이
난 두려워
나보다 더 멋진
그런 남자 니가 만날까봐
아니야 그렇지 않아
정말 너 하나뿐야
속는 셈치고 한번 믿어봐
내 눈에는 너무 이쁜 그녀를
자랑스레 친구에게 보여줬지
그 친구 네게 미안하다며
그녀 얘길 싸 그리다 했지
그녈 만난 많은 남자 중에
내 친구만도 여러명이야
말도 안돼 믿을 수 없어
혹시 쌍둥이
우연히 너를 보았지
다른 남자품안에 너를
한번도 볼 수 없었던
너무 행복한 미소
내 사랑은 무너져 버렸어
그게 아냐 변명이 아니라
그 남자는 나완 상관없어
잠시 나 어지러웠어
기댄 것 뿐이야
날 오해하지 말아 줘
나 역시 많은 여자들
만났다가 헤어져도 봤지만
한꺼번에 많은 여자를
만난 적은 없었어
니가 뭔데 날 아프게 하니
너 때문에 상처돼 버린 내 사랑
이제 다시는 너의 어떤 만남도
나같은 사랑 없을걸
난 두려워 나 역시 다시는
이런 사랑 할수 없을까봐
믿을 수 없겠지만은
니가 첫사랑인데
떠나버리면 어떡하라구
사랑까지는 바라지도 않을께
네 곁에 항상 있게만 해줘
제발 제발
"""]

# input_texts = ["""
# 이제 발표의 마무리야
# """]

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
# 입력 텍스트를 토큰화
inputs = tokenizer(input_texts, padding=True, truncation=True, return_tensors="pt").to(device)
model.to(device)

# 예측 수행
with torch.no_grad():
    outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
    # print(outputs)
    logits = outputs.logits
    probabilities = F.softmax(logits, dim=-1)


# 결과 출력 및 시각화
for i, text in enumerate(input_texts):
    print(f"Input: {text}")
    # print(f"Prediction: {id2label[predictions[i].item()]} (Probability: {probabilities[i][predictions[i]].item():.4f})")

    top_probabilities, top_indices = torch.topk(probabilities[i], 10)
    for j in range(10):
        print(f"Top {j+1} Prediction: {id2label[top_indices[j].item()]} (Probability: {top_probabilities[j].item():.4f})")
    print()

Input: 
너 없는 지금도 눈부신 하늘과
눈부시게 웃는 사람들
나의 헤어짐을 모르는 세상은
슬프도록 그대로인데
시간마저 데려가지 못하게
나만은 널 보내지 못했나봐
가시처럼 깊게 박힌 기억은
아파도 아픈줄 모르고
그대 기억이 지난 사랑이
내 안을 파고 드는 가시가 되어
제발 가라고 아주 가라고
애써도 나를 괴롭히는데
아픈만큼 너를 잊게 된다면
차라리 앓고 나면 그만인데
가시처럼 깊게 박힌 기억은
아파도 아픈 줄 모르고
제발 가라고 아주 가라고
애써도 나를 괴롭히는데
너무 사랑했던 나를
크게 두려웠던 나를
미치도록 너를 그리워했던
날 이제는 놓아줘
보이지 않아 내 안에 숨어
잊으려 하면 할수록 더 아파와
제발 가라고 아주 가라고
애써도 나를 괴롭히는데

Top 1 Prediction: 염세적인 (Probability: 0.0769)
Top 2 Prediction: 마비된 (Probability: 0.0679)
Top 3 Prediction: 외로운 (Probability: 0.0633)
Top 4 Prediction: 우울한 (Probability: 0.0597)
Top 5 Prediction: 눈물이 나는 (Probability: 0.0543)
Top 6 Prediction: 두려운 (Probability: 0.0438)
Top 7 Prediction: 고립된 (Probability: 0.0424)
Top 8 Prediction: 슬픔 (Probability: 0.0386)
Top 9 Prediction: 고립된(당황한) (Probability: 0.0333)
Top 10 Prediction: 안달하는 (Probability: 0.0304)

Input: 옛날 옛날에 남양주시 와부읍
덕소리에서 자란 한 소년이 있었어요
아이는 주머니에 아무것도 없었지만
어머니의 사랑은 가득해
항상 웃음 짓던 소년이였죠
어느 날 그 소년 앞에
금을 두른 부자가 나타났답니다
부자는 그에게 물었어요
너에게 미래를 줄 테니
지금 이 순간을 나와 바꾸지 않겠니?
나 평생 꿈만을 꿨죠


In [17]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

# 입력 텍스트 정의
input_texts = ["""
알잖아 너를 이토록
사랑하며 기다린 나를
뭐가 그리 바쁜지
너무 보기 힘들어
너 도대체 뭐하고 다니니
그게 아냐 이유는 묻지마
그냥 믿고 기다려 주겠니
내게도 사랑을 위한
시간이 필요해
널 받아들일 수 있게
일부로 피하는 거니 노
삐삐쳐도 아무 소식 없는 너
싫으면 그냥 싫다고
솔직하게 말해봐
말리지마 내 이런 사랑을
너만 보면 미칠 것 같은 이맘을
누가 알겠어 웨딩드레스 입은
니 곁에 다른 사람이
난 두려워
나보다 더 멋진
그런 남자 니가 만날까봐
아니야 그렇지 않아
정말 너 하나뿐야
속는 셈치고 한번 믿어봐
내 눈에는 너무 이쁜 그녀를
자랑스레 친구에게 보여줬지
그 친구 네게 미안하다며
그녀 얘길 싸 그리다 했지
그녈 만난 많은 남자 중에
내 친구만도 여러명이야
말도 안돼 믿을 수 없어
혹시 쌍둥이
우연히 너를 보았지
다른 남자품안에 너를
한번도 볼 수 없었던
너무 행복한 미소
내 사랑은 무너져 버렸어
그게 아냐 변명이 아니라
그 남자는 나완 상관없어
잠시 나 어지러웠어
기댄 것 뿐이야
날 오해하지 말아 줘
나 역시 많은 여자들
만났다가 헤어져도 봤지만
한꺼번에 많은 여자를
만난 적은 없었어
니가 뭔데 날 아프게 하니
너 때문에 상처돼 버린 내 사랑
이제 다시는 너의 어떤 만남도
나같은 사랑 없을걸
난 두려워 나 역시 다시는
이런 사랑 할수 없을까봐
믿을 수 없겠지만은
니가 첫사랑인데
떠나버리면 어떡하라구
사랑까지는 바라지도 않을께
네 곁에 항상 있게만 해줘
제발 제발
""","""
마음 울적한 날엔 거리를 걸어보고
향기로운 칵테일에 취해도보고
한편의 시가 있는 전시회장도 가고
밤새도록 그리움에 편질 쓰고파
모차르트 피아노 협주곡 이십일번
그 음악을 내 귓가에 속삭여주며
아침 햇살 눈부심에 나를 깨워줄
그런 연인이 내게 있으면
나는 아직 순수함을 느끼고 싶어
어느 작은 우체국 앞 계단에 앉아
프리지아 꽃 향기를 내게 안겨줄
그런 연인을 만나 봤으면
마음 울적한 날엔 거리를 걸어보고
향기로운 칵테일에 취해도보고
한편의 시가 있는 전시회장도 가고
밤새도록 그리움에 편질 쓰고파
모차르트 피아노 협주곡 이십일번
그 음악을 내 귓가에 속삭여 주며
아침 햇살 눈부심에 나를 깨워줄
그런 연인이 내게 있으면
나는 아직 순수함을 느끼고 싶어
어느 작은 우체국 앞 계단에 앉아
프리지아 꽃향기를 내게 안겨줄
그런 연인을 만나 봤으면
마음 울적한 날엔 거리를 걸어보고
향기로운 칵테일에 취해도보고
한편의 시가 있는 전시회장도 가고
밤새도록 그리움에 편질 쓰고파
창밖에는 우울한 비가 내리고 있어
내 마음도 그 비따라 우울해지네
누가 내게 눈부신 사랑을 가져 줄까
이 세상은 나로 인해 아름다운데
마음 울적한 날엔 거리를 걸어보고
향기로운 칵테일에 취해도보고
한편의 시가 있는 전시회장도 가고
밤새도록 그리움에 편질 쓰고파
""","""
니가 없는 거리에는 내가 할 일이 없어서
마냥 걷다 걷다 보면 추억을 가끔 마주치지
떠오르는 너의 모습 내 살아나는 그리움 한 번에
참 잊기 힘든 사람이란 걸 또 한 번 느껴지는 하루
어디쯤에 머무는지 또 어떻게 살아가는지
걷다 보면 누가 말해줄 것 같아
이 거리가 익숙했던 우리 발걸음이 나란했던
그리운 날들 오늘 밤 나를 찾아온다
널 그리는 널 부르는 내 하루는
애태워도 마주친 추억이 반가워
날 부르는 목소리에 돌아보면
텅 빈 거리 어느새 수많은 니 모습만 가득해
막다른 길 다다라서 낯익은 벽 기대 보면
가로등 속 환히 비춰지는 고백하는 니가 보여
떠오르는 그때 모습 내 살아나는 설레임 한 번에
참 잊기 힘든 순간이란 걸 또 한 번 느껴지는 하루
아직 나를 생각할지 또 그녀도 나를 찾을지
걷다 보면 누가 말해줄 것 같아
이 거리가 익숙했던 우리 발걸음이 나란했던
그리운 날들 오늘 밤 나를 찾아온다
널 그리는 널 부르는 내 하루는
애태워도 마주친 추억이 반가워
날 부르는 목소리에 돌아보면
텅 빈 거리 어느새 수많은 니 모습만 가득해
부풀은 내 가슴이 밤하늘에 외쳐본다
이 거리는 널 기다린다고
널 그리는 널 부르는 내 하루는
애태워도 마주친 추억이 반가워
날 부르는 목소리에 돌아보면
텅 빈 거리 어느새 수많은 니 모습만
가득해
""","""
너와 나 둘이 정신없이 가는 곳
정처 없이 가는 곳, 정해지지 않은 곳
거기서 우리 (whoa-whoa) 서로를 재워주고 (whoa-whoa)
서로를 깨워주고 (whoa-whoa) 서로를 채워주고 (whoa-whoa)
Excuse me 잠시만 아직까진 우린 남
하지만 조만간 중독성을 자랑하는 장난감
지금 이 느낌적인 느낌이 통하는 느낌
녹아버릴 아이스크림
지금이 우리에게는 꿈이야
너와 나 둘이서 추는 춤이야
기분은 미친 듯이 예술이야
Ooh-eh-oh, ooh-eh-oh, ooh-eh-oh
하늘을 날아가는 기분이야
죽어도 상관없는 지금이야
심장은 터질 듯이 예술이야
Ooh-eh-oh, ooh-eh-oh, ooh-eh-oh
예술이야 (예술이야), 예술이야 (예술이야)
예술이야 (예술이야), 이런 날이 올 줄이야
예술이야 (예술이야), 예술이야 (예술이야)
Ooh-eh-oh, ooh-eh-oh, ooh-eh-oh
너와 나 둘이 (whoa-whoa) 밤새 잔을 부딪혀 (whoa-whoa)
밤새 뺨을 부비며 (whoa-whoa) 밤새도록 둘이서 (whoa-whoa)
눈이 점점 풀린다
니도 따라 풀린다
끌린다, 너에 대한 수수께끼가 풀린다
지금 이 춤에 너의 가빠진 숨에
수줍음에 you know what I mean
지금이 우리에게는 꿈이야
너와 나 둘이서 추는 춤이야
기분은 미친 듯이 예술이야
Ooh-eh-oh, ooh-eh-oh, ooh-eh-oh
하늘을 날아가는 기분이야
죽어도 상관없는 지금이야"""]

# input_texts = ["""
# 이제 발표의 마무리야
# """]

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
# 입력 텍스트를 토큰화
inputs = tokenizer(input_texts, padding=True, truncation=True, return_tensors="pt").to(device)
model.to(device)

# 예측 수행
with torch.no_grad():
    outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
    # print(outputs)
    logits = outputs.logits
    probabilities = F.softmax(logits, dim=-1)


# 결과 출력 및 시각화
for i, text in enumerate(input_texts):
    print(f"Input: {text}")
    # print(f"Prediction: {id2label[predictions[i].item()]} (Probability: {probabilities[i][predictions[i]].item():.4f})")

    top_probabilities, top_indices = torch.topk(probabilities[i], 10)
    for j in range(10):
        print(f"Top {j+1} Prediction: {id2label[top_indices[j].item()]} (Probability: {top_probabilities[j].item():.4f})")
    print()

Input: 
알잖아 너를 이토록
사랑하며 기다린 나를
뭐가 그리 바쁜지
너무 보기 힘들어
너 도대체 뭐하고 다니니
그게 아냐 이유는 묻지마
그냥 믿고 기다려 주겠니
내게도 사랑을 위한
시간이 필요해
널 받아들일 수 있게
일부로 피하는 거니 노
삐삐쳐도 아무 소식 없는 너
싫으면 그냥 싫다고
솔직하게 말해봐
말리지마 내 이런 사랑을
너만 보면 미칠 것 같은 이맘을
누가 알겠어 웨딩드레스 입은
니 곁에 다른 사람이
난 두려워
나보다 더 멋진
그런 남자 니가 만날까봐
아니야 그렇지 않아
정말 너 하나뿐야
속는 셈치고 한번 믿어봐
내 눈에는 너무 이쁜 그녀를
자랑스레 친구에게 보여줬지
그 친구 네게 미안하다며
그녀 얘길 싸 그리다 했지
그녈 만난 많은 남자 중에
내 친구만도 여러명이야
말도 안돼 믿을 수 없어
혹시 쌍둥이
우연히 너를 보았지
다른 남자품안에 너를
한번도 볼 수 없었던
너무 행복한 미소
내 사랑은 무너져 버렸어
그게 아냐 변명이 아니라
그 남자는 나완 상관없어
잠시 나 어지러웠어
기댄 것 뿐이야
날 오해하지 말아 줘
나 역시 많은 여자들
만났다가 헤어져도 봤지만
한꺼번에 많은 여자를
만난 적은 없었어
니가 뭔데 날 아프게 하니
너 때문에 상처돼 버린 내 사랑
이제 다시는 너의 어떤 만남도
나같은 사랑 없을걸
난 두려워 나 역시 다시는
이런 사랑 할수 없을까봐
믿을 수 없겠지만은
니가 첫사랑인데
떠나버리면 어떡하라구
사랑까지는 바라지도 않을께
네 곁에 항상 있게만 해줘
제발 제발

Top 1 Prediction: 외로운 (Probability: 0.2201)
Top 2 Prediction: 안달하는 (Probability: 0.0580)
Top 3 Prediction: 두려운 (Probability: 0.0511)
Top 4 Prediction: 질투하는 (Probability: 0.0496)
Top 5 Prediction: 눈물이 나는 (Probability: 0.0482)
Top 6 Prediction: 염세적인 (Probability: 0.0425

In [172]:
tokenizer.vocab_size

32000

In [177]:
model_path = "./struc3_models_2"

model.save_pretrained(model_path)
model.save_config(model_path + 'config.json')
tokenizer.save_pretrained(model_path)

('./struc3_models_2/tokenizer_config.json',
 './struc3_models_2/special_tokens_map.json',
 './struc3_models_2/vocab.txt',
 './struc3_models_2/added_tokens.json',
 './struc3_models_2/tokenizer.json')

In [137]:
import torch
import numpy as np
from transformers import pipeline

# Set seed for reproducibility
# torch.manual_seed(0)
# np.random.seed(0)

# Initialize the classifier
classifier = pipeline("text-classification", model="dudududukim/klue-bert-base-senti")

# Classify text
result = classifier("이게 돼야해")
print(result)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dudududukim/klue-bert-base-senti and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'label': 'LABEL_58', 'score': 0.03276543319225311}]
